In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다', '어떻다'] 

In [3]:
# 품사에 따라 
tokenizer = Okt()

def tokenize(text):
    raw_pos_tagged = tokenizer.pos(text)
    filtered = [word for word, pos in raw_pos_tagged 
                if pos in ['Noun', 'Verb', 'Adjective'] and word not in del_list]
    
    return filtered

In [4]:
df = pd.read_excel('result_231012_1218.xlsx')
df = df.dropna()
df.head()

,Date,Category,Press,Title,Document,Link
0,2023.10.12. 오전 10:11,정치 일반,MBC,"[시선집중] 이주호 “사교육 부담 증가는 학원가 불안 마케팅.. 올 수능, 킬러문항...","<이주호 사회부총리 겸 교육부장관>\r-대입 제도 개편.. 안정과 공정, 혁신에 최...",https://n.news.naver.com/mnews/article/214/000...
1,2023.10.12. 오전 10:11,정치 일반,세계일보,"대통령실, 보선 참패에 “어떤 선거 결과든 엄중히 받아들여야”","국민의힘, 13일 오전 긴급최고위원회의 개최 “당 체질 개선 논의” 김기현 국...",https://n.news.naver.com/mnews/article/022/000...
2,2023.10.12. 오전 10:09,정치 일반,TV조선,"北, 김영호 통일장관에 ""미치광이·매국 역적"" 막말",북한의 대외 선전매체들이 일제히 김영호 통일부 장관에 대해 막말을 동원하며 비난에 ...,https://n.news.naver.com/mnews/article/448/000...
3,2023.10.12. 오전 10:08,정치 일반,연합뉴스,선서하는 윤희근 경찰청장,(서울=연합뉴스) 김도훈 기자 = 12일 오전 서울 경찰청에서 열린 경찰청·도로교통...,https://n.news.naver.com/mnews/article/001/001...
4,2023.10.12. 오전 10:08,정치 일반,뉴시스,"귀어귀촌 인구유입 사업 예산 전액 삭감…""어촌소멸 나 몰라라""","도시민 어촌유치지원사업·귀어학교 개설사업 내년도 예산 전액 삭감소병훈 위원장 ""핵오...",https://n.news.naver.com/mnews/article/003/001...


In [5]:
# 카테고리 Label Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
df['Encoded_Category'] = le.fit_transform(df['Category'])

X = df['Title'] + " " + df['Document']
Y = df['Encoded_Category']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [6]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, lowercase=False)

X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

### Multinomial - Naive Bayes

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

classifier = GaussianNB();
classifier.fit(X_train_vec, Y_train)

# predicting test set results
Y_pred = classifier.predict(X_test_vec)
print('Accuracy score: ', accuracy_score(Y_test, Y_pred))

MSE = mean_squared_error(Y_test, Y_pred)
print('Mean Squared Error: ', MSE)

# making the confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print(cm)

Accuracy score:  0.6828721017202692
Mean Squared Error:  1.2094240837696335
[[188  26   9  29   9]
 [ 40 162  31  42  18]
 [ 19  32 148  44  32]
 [  9   7  17 185   9]
 [  8  14  17  12 230]]


### Getting Result

In [11]:
# 사용자가 데이터를 입력, 전처리
input_data = input("텍스트를 입력하세요: ")
raw_data = tokenize(input_data)
input_data_vec = vectorizer.transform(raw_data).toarray()

# 결과 출력_Naive Bayes
dt_pred = classifier.predict(input_data_vec)
pred_res = le.inverse_transform(dt_pred)
print("Naive Bayes를 통해 얻은 결과는 '", pred_res[0], "'입니다.")


텍스트를 입력하세요: 태풍'독수리'북상 중..."한반도 장마 영향 가능성"
Naive Bayes를 통해 얻은 결과는 ' 사회 일반 '입니다.
